<a href="https://colab.research.google.com/github/Confirmation-Bias-Analyser/Data-Collection/blob/main/Twitter_Data_Collection_from_PHEME_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip3 install anytree

# Import essential libraries

In [ ]:
import tensorflow as tf
import pandas as pd
from google.colab import files
from anytree import Node, RenderTree, search
import random
import json

# The shutil module offers a number of high-level 
# operations on files and collections of files.
import os
import shutil
import json

# Declare important functions

In [ ]:
import re
import urllib
from urllib.parse import urlparse

def createTweetsTree(dictionary, tree_root):
    for key, item in dictionary.items():
        child = Node(key, parent=tree_root)

        if len(dictionary[key]) != 0:      
            createTweetsTree(dictionary[key], child)

        else:
            continue

def processTweetsToDict(conversation_data):
    conversation_dict = {'id':[], 'timestamp':[], 'reply_to':[], 'comment':[]}

    for i in conversation_data:
        print('User ID:', i['id'], 
              'Time:', i['user']['created_at'])
        print('In reply to:', i['in_reply_to_status_id'])
        print(i['text'], '\n')

        conversation_dict['id'].append(i['id'])
        conversation_dict['timestamp'].append(i['user']['created_at'])
        conversation_dict['reply_to'].append(i['in_reply_to_status_id'])
        conversation_dict['comment'].append(i['text'])

    return conversation_dict

def getLinks(string):
    urls = re.findall("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", string)
    links = ''

    for url in urls:
        try:
            opener = urllib.request.build_opener()
            request = urllib.request.Request(url)
            response = opener.open(request)
            actual_url = response.geturl()
                      
            if '](' in actual_url:
                actual_url = actual_url.split('](')[0]
          
            links += actual_url + ';'
            
            
        except:
            if '](' in url:
                url = url.split('](')[0]
          
            links += url + ';'

    return links

def getURLfromList(url):
    if ';' in url:
        url = url.split(';')[:-1]
        result = []
    
        for i in url:
            result.append(urlparse(i).hostname)
        
        return result

    else:
        return ''   

# Extract data from PHEME Dataset

In [ ]:
URL = "https://figshare.com/ndownloader/files/11767817/PHEME_veracity.tar.bz2"

dataset = tf.keras.utils.get_file(fname="PHEME_veracity.tar.bz2", 
                                  origin=URL,
                                  untar=True,
                                  cache_dir='.',
                                  cache_subdir='')

In [ ]:
# Unzip extracted file
!tar xzf '/content/PHEME_veracity.tar.bz2.tar.gz'

# Import extracted dataset and understand directory

In [ ]:
threads = []
data = ['non-rumours', 'rumours']

for i in os.listdir('/content/all-rnr-annotated-threads'):
  if i[0] != '.':
    print(i)
    threads.append(i)

ferguson-all-rnr-threads
germanwings-crash-all-rnr-threads
gurlitt-all-rnr-threads
ottawashooting-all-rnr-threads
sydneysiege-all-rnr-threads
putinmissing-all-rnr-threads
ebola-essien-all-rnr-threads
charliehebdo-all-rnr-threads
prince-toronto-all-rnr-threads


Understand the number of files in each sub-directory.

In [ ]:
rumours = 0
non_rumours = 0

for i in threads:
  path = '/content/all-rnr-annotated-threads/' + i
  print(i)

  for j in os.listdir(path):
    
    for k in data:

      for l in os.listdir(path + '/' + k):
        if k == data[0] and l[0] != '.':
          non_rumours += 1

        elif k == data[1] and l[0] != '.':
          rumours += 1
  
  print('Rumours:', rumours)
  print('Non-rumours:', non_rumours)
  print()

ferguson-all-rnr-threads
Rumours: 1704
Non-rumours: 5154

germanwings-crash-all-rnr-threads
Rumours: 3132
Non-rumours: 6540

gurlitt-all-rnr-threads
Rumours: 3498
Non-rumours: 7002

ottawashooting-all-rnr-threads
Rumours: 6318
Non-rumours: 9522

sydneysiege-all-rnr-threads
Rumours: 9450
Non-rumours: 13716

putinmissing-all-rnr-threads
Rumours: 10206
Non-rumours: 14388

ebola-essien-all-rnr-threads
Rumours: 10290
Non-rumours: 14388

charliehebdo-all-rnr-threads
Rumours: 13038
Non-rumours: 24114

prince-toronto-all-rnr-threads
Rumours: 14412
Non-rumours: 24138



# Read sample file from sub-directory

In [ ]:
sample_directory = f'/content/all-rnr-annotated-threads/{random.choice(threads)}/{random.choice(data)}/'

reactions = []

for i in os.listdir(sample_directory):
  if i[0] != '.':
    sample_directory_reactions = sample_directory + i + '/reactions/'

    for j in os.listdir(sample_directory_reactions):
      if j[0] != '.':
        reactions.append(j)

    if len(reactions) < 1:
      reactions = []
      continue
      
    else:
      break

print(sample_directory_reactions)

/content/all-rnr-annotated-threads/ferguson-all-rnr-threads/rumours/500398504075071488/reactions/


Tweets are saved as JSON files in the sub-directories. We are able to view the attributes of the JSON file.

In [ ]:
json_file = sample_directory_reactions + random.choice(reactions)
print(json_file)

with open(json_file, 'r') as j:
     contents = json.loads(j.read())

contents

/content/all-rnr-annotated-threads/ferguson-all-rnr-threads/rumours/500398504075071488/reactions/500437107043627009.json


{'contributors': None,
 'coordinates': None,
 'created_at': 'Sat Aug 16 00:21:18 +0000 2014',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [],
  'user_mentions': [{'id': 977200838,
    'id_str': '977200838',
    'indices': [0, 12],
    'name': 'Blake Wood',
    'screen_name': 'blakewood24'},
   {'id': 14560097,
    'id_str': '14560097',
    'indices': [13, 19],
    'name': 'Vaughn Sterling',
    'screen_name': 'vplus'},
   {'id': 93129160,
    'id_str': '93129160',
    'indices': [20, 34],
    'name': 'nick wright',
    'screen_name': 'getnickwright'}]},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 500437107043627009,
 'id_str': '500437107043627009',
 'in_reply_to_screen_name': 'Royalsoptimist5',
 'in_reply_to_status_id': 500436755749695488,
 'in_reply_to_status_id_str': '500436755749695488',
 'in_reply_to_user_id': 1636077204,
 'in_reply_to_user_id_str': '1636077204',
 'lang': 'en',
 'place': None,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a

# Read sample directory for its tweets and comments

In [ ]:
sample_directory

'/content/all-rnr-annotated-threads/ferguson-all-rnr-threads/rumours/'

## All tweets in sample directory

In [ ]:
count = 0

for i in os.listdir(sample_directory):
  if i[0] != '.':
    json_file = sample_directory + '/' + i + '/source-tweets/' + i + '.json'

    with open(json_file, 'r') as j:
      contents = json.loads(j.read())

    print(contents['text'], '\n')
  
  count += 1
  
  if count == 20:
    break

Is the Police Chief in #Ferguson changing the story?  https://t.co/8ujqp0lpb5 

Lets bomb nurseries w/ White Babies!? FBI: #NBP leader inciting violence in #Ferguson http://t.co/TCXSbfrTsV #tcot http://t.co/XDZqcWnoUQ 

#Ferguson chief said the officer was unaware of any robbery &amp; stopped #MikeBrown for jay walking. #TheJig http://t.co/uLWzIJH5WJ 

It's not #MikeBrown in those #Ferguson police video stills. Not unless he changed his shoes after the robbery but before he met the officer. 

Not just Mike Brown… Second police shooting in #Ferguson leaves man in critical condition http://t.co/uZpgcJK0bB 

Prayers out to mya, HU graduate who was shot while protesting in St. Louis! #MikeBrown #Ferguson http://t.co/J0OEIkJLHU 



## Look at directory within each tweet

In [ ]:
count = 0

for i in os.listdir(sample_directory):
  if i[0] != '.':
    print(i, '-', os.listdir(sample_directory + '/' + i))

  count += 1
  if count == 20:
    break

500398504075071488 - ['annotation.json', 'source-tweets', '._annotation.json', '._reactions', '._structure.json', 'structure.json', '._source-tweets', 'reactions']
499648409767538688 - ['annotation.json', 'source-tweets', '._annotation.json', '._reactions', '._structure.json', 'structure.json', '._source-tweets', 'reactions']
500364290545958912 - ['annotation.json', 'source-tweets', '._annotation.json', '._reactions', '._structure.json', 'structure.json', '._source-tweets', 'reactions']
500285732578013185 - ['annotation.json', 'source-tweets', '._annotation.json', '._reactions', '._structure.json', 'structure.json', '._source-tweets', 'reactions']
499579905508917248 - ['annotation.json', 'source-tweets', '._annotation.json', '._reactions', '._structure.json', 'structure.json', '._source-tweets', 'reactions']
499642374428307457 - ['annotation.json', 'source-tweets', '._annotation.json', '._reactions', '._structure.json', 'structure.json', '._source-tweets', 'reactions']


## Look at the structure of tweets and its comments within the sample directory

In [ ]:
count = 0
allTweets = []

for i in os.listdir(sample_directory):
  if i[0] != '.':
    allTweets.append(i)
    reactions_path = sample_directory + '/' + i + '/structure.json'
    
    with open(reactions_path, 'r') as k:
      contents = json.loads(k.read())

    print(count, contents, '\n')
    count += 1
  
  if count == 20:
    break

0 {'500398504075071488': {'500398827103608832': [], '500399103746908161': [], '500399338368294913': [], '500399431674781698': [], '500399480999403520': [], '500399523165130752': [], '500399671282388992': [], '500399693256744961': [], '500400192772792321': [], '500400265787232257': {'500406508685762560': {'500406871476297728': {'500408124801445889': {'500408569049513985': {'500409961092886528': [], '500411105517117440': []}}}}}, '500403389310959616': [], '500404069220229120': {'500405855523983361': []}, '500406744141791233': [], '500408206242238466': [], '500418868993880064': [], '500419596885983233': [], '500422416951029760': {'500422884964040704': []}, '500422793595326466': {'500432042967203840': {'500432740789940224': {'500435050673213440': {'500436755749695488': {'500437107043627009': [], '500443511775576064': {'500449194885124096': {'500450613264920576': {'500452100019482624': [], '500452379720818688': {'500453593485701120': {'500455598710722561': {'500455678436061185': {'500456294

## All the comments of each tweet

In [ ]:
count = 0

for i in os.listdir(sample_directory):
  if i[0] != '.':

    reactions_path = sample_directory + '/' + i + '/reactions/'
    print(i)

    for j in os.listdir(reactions_path):
      if j[0] != '.':
        with open(reactions_path + j, 'r') as k:
          contents = json.loads(k.read())

        print(contents['id'], contents['text'], contents['in_reply_to_status_id'], '\n')

    count += 1
    if count == 3:
      break

500398504075071488
500437107043627009 @blakewood24 @vplus @getnickwright sorry meant anyone not anymore. 500436755749695488 

500411105517117440 @CartwheelPrint the availability heuristic applies to the cops decision making and now the crowds here is summary http://t.co/5GSGFBYfWt 500408569049513985 

500419596885983233 @vplus about this kid? http://t.co/3VoHqJGBMn 500398504075071488 

500453593485701120 @Royalsoptimist5 So in conclusion, you're a selfish ass hole who doesn't care about anyone else's life but your own. Got it. Enjoy your day. 500452379720818688 

500452379720818688 @blakewood24 so honestly long as it doesn't effect me or my life than no doesn't really bother me if other people have issues. 500450613264920576 

500443511775576064 @Royalsoptimist5 @vplus @getnickwright People are being shot, tear gassed, &amp; unlawfully detained in the streets. That's not constitutional. 500436755749695488 

500399693256744961 @vplus @rcinstitute Yes, and it will change a few more times

# Select random tweet and read the tweet

In [ ]:
wanted_id = random.choice(allTweets)
print(wanted_id, '\n')

500247666924986369 



## Visualise tree of comments

In [ ]:
with open(sample_directory + '/' + wanted_id + '/structure.json', 'r') as k:
  contents = json.loads(k.read())

contents

{'500247666924986369': {'500247911246987264': [],
  '500252377140977664': [],
  '500254112399630339': [],
  '500255271042949120': [],
  '500256828228632576': [],
  '500259324556738560': [],
  '500260366807752704': [],
  '500261138534125568': [],
  '500262399388049409': [],
  '500266589636141057': [],
  '500269705715003392': [],
  '500270246478249984': {'500276146223607808': {'500276471781662720': []}},
  '500272554201075713': [],
  '500276272153391107': {'500277642084364289': []}}}

In [ ]:
root = Node(wanted_id)
createTweetsTree(contents[wanted_id], root)

tweet_path = sample_directory + '/' + wanted_id
allComments = []

with open(tweet_path + '/source-tweets/' + wanted_id + '.json', 'r') as j:
  master_tweet= json.loads(j.read())

print(master_tweet['text'], '\n')

for j in os.listdir(tweet_path + '/reactions/'):
  if j[0] != '.':
    
    with open(tweet_path + '/reactions/' + j, 'r') as k:
      contents = json.loads(k.read())
      allComments.append(contents)

KKK already raising money for the police officer who shot and killed #MikeBrown - #Ferguson http://t.co/VWkZXjHF7V 



In [ ]:
for pre, fill, node in RenderTree(root):
  print("%s%s" % (pre, node.name))

500247666924986369
├── 500247911246987264
├── 500252377140977664
├── 500254112399630339
├── 500255271042949120
├── 500256828228632576
├── 500259324556738560
├── 500260366807752704
├── 500261138534125568
├── 500262399388049409
├── 500266589636141057
├── 500269705715003392
├── 500270246478249984
│   └── 500276146223607808
│       └── 500276471781662720
├── 500272554201075713
└── 500276272153391107
    └── 500277642084364289


## Convert comments to dataframe

In [ ]:
result = processTweetsToDict(allComments)

User ID: 500276272153391107 Time: Fri Sep 18 15:05:59 +0000 2009
In reply to: 500247666924986369
@TheAnonMessage2 That's Sweet, but a bad investment. If #MichealBrownKiller accepts #KKK money it'll be bad PR for him &amp; #FergusonPolice 

User ID: 500255271042949120 Time: Wed Jun 10 16:34:40 +0000 2009
In reply to: 500247666924986369
@TheAnonMessage2 @CriticalDragon1 Of course they are 😡💂 

User ID: 500247666924986369 Time: Tue Aug 12 06:41:41 +0000 2014
In reply to: None
KKK already raising money for the police officer who shot and killed #MikeBrown - #Ferguson http://t.co/VWkZXjHF7V 

User ID: 500256828228632576 Time: Wed May 06 05:25:14 +0000 2009
In reply to: 500247666924986369
Dont believe the lies.  They are trying to stir hate! #Ferguson  http://t.co/D8gyIU7fsr @TheAnonMessage2 #MikeBrown 

User ID: 500259324556738560 Time: Sun Jun 06 00:05:06 +0000 2010
In reply to: 500247666924986369
@TheAnonMessage2 @CriticalDragon1 When the KKK applauds your actions, that's a pretty good cl

In [ ]:
df = pd.DataFrame.from_dict(result)
df['id'] = df['id'].astype(str)
df['reply_to'] = df['reply_to'].astype(str)
df['url'] = df['comment'].apply(lambda x: getLinks(x))
df['link_title'] = df['url'].apply(lambda x: getURLfromList(x))

df.head()

,id,timestamp,reply_to,comment,url,link_title
0,500276272153391107,Fri Sep 18 15:05:59 +0000 2009,5.002476669249864e+17,"@TheAnonMessage2 That's Sweet, but a bad inves...",,
1,500255271042949120,Wed Jun 10 16:34:40 +0000 2009,5.002476669249864e+17,@TheAnonMessage2 @CriticalDragon1 Of course th...,,
2,500247666924986369,Tue Aug 12 06:41:41 +0000 2014,nan,KKK already raising money for the police offic...,http://littlegreenfootballs.com/page/310691_Ha...,[littlegreenfootballs.com]
3,500256828228632576,Wed May 06 05:25:14 +0000 2009,5.002476669249864e+17,Dont believe the lies. They are trying to sti...,https://pastebin.com/d19hGj4A;,[pastebin.com]
4,500259324556738560,Sun Jun 06 00:05:06 +0000 2010,5.002476669249864e+17,@TheAnonMessage2 @CriticalDragon1 When the KKK...,,
